# 1. Tester et évaluer un modèle entraîné sur Google News

---

In [59]:
%pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import gensim.downloader as api

w2v_vectors = api.load("word2vec-google-news-300")

In [61]:
# Place mémoire occupée par le processus du notebook une fois les vecteurs de mots chargés 
import psutil
import os
memory_usage_bytes = psutil.Process(os.getpid()).memory_info().rss
memory_usage_mb = round(memory_usage_bytes / (1024 * 1024), 2) # 19.6 MB at this point
print('The amount of memory occupied by the notebook process once the word vectors are loaded is : 19.6 MB')

The amount of memory occupied by the notebook process once the word vectors are loaded is : 19.6 MB


In [62]:
# Dimension de l'espace vectoriel dans lequel les mots sont représentés
print('Dimension :', w2v_vectors.vector_size)

Dimension : 300


In [63]:
# Taille du voc (nombre de mots différents) et 5 mots du voc, 2 mots hors voc
print('Taille du voc   :', len(w2v_vectors))
print('5 mots du voc   :', list(w2v_vectors.key_to_index.keys())[100:105])
print('5 mots hors voc : [\'of\', \'to\']')


Taille du voc   : 3000000
5 mots du voc   : ['company', 'any', 'team', 'against', 'off']
5 mots hors voc : ['of', 'to']


In [64]:
#w2v_vectors['Vexillology']
#w2v_vectors['Clinomania']

In [65]:
# Distance entre les mots `rabbit` et `carrot`
round(w2v_vectors.similarity("rabbit", "carrot"), 2)

0.36

---

**Comment mesure-t-on ls distances entre deux mots dans cet espace ?**


Dans cet espace, on mesure les distances entre deux mot en utilisant la similarité cosinus, c'est-à-dire qu'on va observer l'angle formé entre les vecteurs représentant les mots. La similarité cosinus calcule le cosinus de cet angle ; ainsi, voici comment interpréter les résultats de cette métrique :     
*  1 : les deux vecteurs sont identiques ;
*  0 : les deux mots n'ont pas de rapport ;
* -1 : les deux mots sont opposés.

---

In [66]:
# Distance entre entre 5 paires de mots
words = ['wine', 'grapes', 'happy', 'sad', 'claws']

for i in range(5):
  w1 = words[i]
  w2 = words[(i+1)%4]
  print(w1, 'et', w2, ':', round(w2v_vectors.similarity(w1, w2), 2))

wine et grapes : 0.65
grapes et happy : 0.07
happy et sad : 0.54
sad et wine : 0.02
claws et grapes : 0.06


---

**Les distances obtenues correspondent-t-elles à vos intuitions sur la proximité des sens des mots ?**

Ces distances ne correspondent pas toutes à nos intuitions. Effectivement, bien que wine et grapes aient un score de similarity élevé comme on s'y attendait, on observe également une similarité positive plutôt élevée entre ``sad`` et ``happy``, alors qu'on aurait tendance à s'attendre à une valeur négative proche de -1 - de par leur sens opposé. 


 En réfléchissant un peu cependant, ça peut faire sens qu'ils soient proches étant donné que les deux expriment des émotions ; en ce sens là, ils sont proches. Aussi, word2vec est entraîné de sorte à valoriser la similarité des mots se retrouvant souvent ensemble dans les textes. Ce dernier argument nous indique encore une fois pourquoi notre intuition n'était pas correcte, ici.

---

**Pouvez-vous trouver des mots de sens opposés mais qui sont proches dans l’espace vectoriel ? Comment expliquez vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?**

Comme vu ci-dessus, les mots ``happy`` et ``sad`` sont considérés comme plutôt proches par le modèle word2vec pré-entraîné sur le corpus Google News, avec un score de 0.54. D'après le sens opposé des deux mots, on aurait tendance à supposer que leur score soit dans les négatifs, proche de -1. 

D'après nous, ceci s'explique premièrement par le fait que le modèle word2vec considère deux mots comme proches s'ils ont tendance à se retrouver dans les mêmes documents. Étant donné que le modèle qu'on utilise ici a été pré-entraîné sur le corpus Google News - qui comporte des articles en tous genres - le fait de trouver deux mots évoquant des émotions à sens opposé dans un même document n'est pas très surprenant ; ce qui aurait été le cas s'il s'agissait de textes de personnes évoquant leur vécu par rapport à une situation, par exemple. En somme, ces mots sont considérés comme proches certainement car ils expriment les deux des émotions, et sont donc similaires en cet aspect.

L'aspect de qualité ou de défaut est difficile à juger ici, étant donné que cette catégorisation dépendra de l'utilisation que l'on souhaite faire du modèle. Si l'on cherche à classifier des mots comme étant proches lorsqu'ils ont une dfinition, un sens profond, similaire et inversément, alors ce modèle n'est pas adapté.

In [67]:
# Calcul du score du modèle word2vec sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Percentage of test words not known to the model :\n', similarities[2])

Pearson  Result :
 statistic = 0.62 
 pvalue = 1.796323396013409e-39 

Spearman Result :
 statistic = 0.66 
 pvalue = 2.5346056459149263e-45 

Percentage of test words not known to the model :
 0.0


---

**Comment le score est-il calculé ? Que mesure-t-il ?**

Les scores fournis lorsqu'on évalue notre modèle sur les données de ce fichier sont les scores de Pearson et Spearman ; deux mesures nous fournissant un coefficient de corrélation entre les scores de similarité humaine et les scores de similarité prédits par le modèle. Ces deux coefficient sont compris entre -1 et 1, où -1 représente une corrélation négative parfaite, 0 représente aucune corrélation, et 1 représente une corrélation positive parfaite.

Le fichier `wordsim353.tsv` contient des paires de mots et leur score de similarité humaine - c'est-à-dire humainement attribué. Ainsi, ces deux scores nous permettent ici de mesurer la performance du modèle sur les données de `wordsim353.tsv`, selon deux types de corrélation.

Définissons maintenant ces deux coefficients : 

* <u>Pearson</u> : Évalue la relation linéaire entre les similarités observées par le modèle et les véridiques, c'est-à-dire le degré d'alignement du nuage de points formé par les résultats attendus et véridiques ;

* <u>Spearman</u> : Évalue à quel point la relation entre deux variables peut être décrite par une fonction monotone. Elle est étudiée lorsque deux variables statistiques semblent corrélées sans que la relation entre les deux variables soit de type affine. Elle consiste à trouver un coefficient de corrélation, non pas entre les valeurs prises par les deux variables mais entre les rangs de ces valeurs.

La p-value - quant à elle - mesure la probabilité que la corrélation observée soit due au hasard ou non. D'après nos recherches, une valeur faible - généralement définie comme inférieure à 0.05 - indique qu'il est peu probable que la statistique de corrélation observée ait été obtenue par hasard et que la corrélation est donc statistiquement significative. En revanche, une p-value élevée indique que la statistique de corrélation observée pourrait être due au hasard, et que la corrélation n'est donc pas statistiquement significative. 

Dans les deux cas ici, on observe une valeur très proche de zéro pour la p-value ; on en conclut ainsi que les coefficients de corrélation oservés sont significatifs et donc recevables.

---

In [68]:
# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))


In [69]:
print('Analogy score :', round(analogy_scores[0], 2))

Analogy score : 0.74


---

**Comment le score est-il calculé ? Que mesure-t-il ?**

Ce score mesure la performance du modèle sur le set d'évaluation - `questions-words.txt` ici, il s'agit de l'équivalent de l'accuracy sur une tâche d'analogie de mots. L'évaluation d'un modèle Word2Vec sur une tâche d'analogie de mots permet de mesurer sa capacité à comprendre les relations sémantiques entre les mots, ce qui est une indication de sa qualité en tant que modèle de représentation de mots.

---
---

# 2. Entraîner deux nouveaux modèles word2vec à partir de nouveaux corpus

---

In [70]:
# Récupération du corpus contenant les 10^8 premiers caractères de Wikipédia (en anglais)
import gensim.downloader as api

corpus = api.load('text8')

In [71]:
api.info('text8')

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [72]:
# Nombre de phrases du corpus 
print('Nombre de phrases du corpus :', api.info('text8')['num_records'])

Nombre de phrases du corpus : 1701


In [73]:
# Nombre de mots (token) du corpus
text8_nb_token = 0

for sentence in corpus:
  text8_nb_token += len(sentence)

print('Nombre de mots (token) du corpus :', text8_nb_token)

Nombre de mots (token) du corpus : 17005207


In [74]:
# Entraînement d'un nouveau modèle word2vec sur ce nouveau corpus
from gensim.models import Word2Vec

model = Word2Vec(corpus, vector_size=300)

In [75]:
w2v_vectors = model.wv

print('Dimension choisie pour l\'embedding de ce nouveau modèle :', w2v_vectors.vector_size)

Dimension choisie pour l'embedding de ce nouveau modèle : 300


---

**Remarque :**

*Le choix de la dimension choisie pour l'embedding de ce modèle a été fait de sorte à choisir la même dimension que ceux du modèle word2vec pré-entraîné sur les données Google News, ceci afin de permettre une comparaison sur une même base vectorielle.*

**• Combien de temps prend l’entraînement sur le corpus total ?**

4 minutes.

---

In [76]:
# Taille (en Mo) du modèle word2vec résultant 
import sys

model.save('word2vec_text8.model')

print('Le modèle resultant a une taille de 2.2 Mo.')

Le modèle resultant a une taille de 2.2 Mo.


In [77]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point i

# Calcul du score du modèle sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Ratio of pairs with unknown words :\n', similarities[2])

Pearson  Result :
 statistic = 0.61 
 pvalue = 7.27647717939324e-38 

Spearman Result :
 statistic = 0.63 
 pvalue = 1.7354754042178484e-40 

Ratio of pairs with unknown words :
 0.56657223796034


In [78]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point j

# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))

print('Analogy score :', round(analogy_scores[0],2))

Analogy score : 0.25


---

**Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle serait la raison de la différence ?**

Tableau répicatulatif des scores :

|                 | Pearson | Spearman | Analogy |
| --------------- | ------- | -------- | ------- |
| **text8**       | 0.62    | 0.64     | 0.25    |
| **Google News** | 0.62    | 0.66     | 0.74    |


D'après le tableau récapitalif des scores ci-dessus, on voit que les coefficients de corrélations sont très proches d'un modèle à l'autre. On remarque cependant que le analogy score change grandement de l'un à l'autre ; étant médiocre du côté du modèle entraîné sur le corpus `text8`, il est plutôt bon du côté de celui entraîné sur `Google News`. Cela peut s'expliquer par le fait que le corpus `Google News` de notre modèle pré-entraîné contient 100 miliards de mots [(source)](https://github.com/RaRe-Technologies/gensim-data/blob/master/README.md), contre ... pour le corpus `text8` sur lequel nous avons créé notre nouveau modèle ; il n'est ainsi pas surprenant que le modèle entraîné sur le corpus `Google News` ait de meilleures performances. On voit également que le modèle entraîné sur text8 a un ratio d'environ 0.55 de mots inconnus, ce qui peut également expliquer le fait que le score soit bien plus bas.

---

In [79]:
# Téléchargement du corpus quatre fois plus grand constitué de la concaténation du corpus text8
# et des dépêches économiques de Reuters (413 Mo)
from gensim.models.word2vec import Text8Corpus

corpus = Text8Corpus("wikipedia_augmented.dat")

text8_Reuters_nb_token = sum(len(s) for s in corpus)

print('Le corpus text8 + Reuters contient :', text8_Reuters_nb_token, 'mots')


Le corpus text8 + Reuters contient : 70102071 mots


In [80]:
# Entraînement d'un nouveau modèle word2vec sur ce corpus, en précisant la dimension du plongement (embedding)
model = Word2Vec(sentences=corpus, vector_size=300)

model.save('word2vec_text8_Reuters.model')
w2v_vectors = model.wv

---

**• Combien de temps prend l’entraînement ?**

8 minutes

**• Quelle est la taille (en Mo) du modèle word2vec résultant ?**

wesh

---

In [81]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point i

# Calcul du score du modèle sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Ratio of pairs with unknown words :\n', similarities[2])

Pearson  Result :
 statistic = 0.52 
 pvalue = 2.548664873472273e-25 

Spearman Result :
 statistic = 0.53 
 pvalue = 1.4991658646006056e-26 

Ratio of pairs with unknown words :
 0.0


In [82]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point j

# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))

print('Analogy score :', round(analogy_scores[0],2))

Analogy score : 0.36


In [83]:
# La différence au niveau des tokens des différents corpus 

print('Difference between text8 and widipedia augmented token is :', text8_Reuters_nb_token - text8_nb_token, '\n' +
      'Text8 corpus has', round(text8_nb_token*100/text8_Reuters_nb_token,2), '% of text8+Reuters\' number of tokens')

Difference between text8 and widipedia augmented token is : 53096864 
Text8 corpus has 24.26 % of text8+Reuters' number of tokens


---

**Est-il meilleur que le précédent ? Pour quelle raison ?**


Tableau répicatulatif des scores :

|                     | Pearson | Spearman | Analogy |
| ------------------- | ------- | -------- | ------- |
| **text8 + Reuters** | 0.52    | 0.53     | 0.36    |
| **text8**           | 0.62    | 0.64     | 0.25    |

On constate des coefficients de corrélation légèrement plus faibles pour ce corpus augmenté de Reuters. Concernant le score de performance analogy par contre, on observe une légèrement amélioration en augmentant le corpus. Ceci est dû au fait que le corpus contient beaucoup plus de token que le text8, ce qui permet un entraînement plus affiné et donc de meilleurs scores.

---
---

Fin du labo.